In [6]:
import cv2
import numpy as np

def get_plant_height(image_path, reference_height_cm, reference_pixel_height):
    """Detects plant height in pixels and converts it to centimeters using a reference object."""
    frame = cv2.imread(image_path)

    if frame is None:
        print(f"Error loading image: {image_path}")
        return None

    # Resize image to standard dimensions for consistent processing
    frame = cv2.resize(frame, (800, 600))

    # Convert image to HSV color space for better color-based segmentation
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define HSV range to isolate green regions (typical for plant leaves)
    lower_color = np.array([30, 40, 40])
    upper_color = np.array([90, 255, 255])
    mask_color = cv2.inRange(img_hsv, lower_color, upper_color)

    # --- Thresholding Section ---

    # Convert to grayscale for intensity-based thresholding
    gray_for_thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Adaptive thresholding handles varying lighting conditions in the image.
    # It isolates plant structure based on local intensity differences.
    adaptive_thresh = cv2.adaptiveThreshold(
        gray_for_thresh, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV,
        blockSize=15,
        C=5
    )

    # Binary thresholding on the color mask ensures crisp segmentation (binary 0/255)
    _, mask_thresh = cv2.threshold(mask_color, 127, 255, cv2.THRESH_BINARY)

    # --- Edge Detection Section ---

    # Apply Gaussian blur to reduce noise before edge detection
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)

    # Convert blurred image to grayscale for edge detection
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

    # Use Canny edge detection to find strong edges (plant boundaries)
    edges = cv2.Canny(gray, 50, 150)

    # --- Combine Masks ---

    # Combine thresholded color mask and edge detection to form final binary mask
    # This hybrid mask improves robustness by using both color and intensity cues
    mask_combined = cv2.bitwise_or(mask_thresh, edges)

    # --- Morphological Processing ---

    # Use morphological operations to clean up noise and close gaps
    kernel = np.ones((5, 5), np.uint8)
    mask_combined = cv2.morphologyEx(mask_combined, cv2.MORPH_OPEN, kernel)
    mask_combined = cv2.morphologyEx(mask_combined, cv2.MORPH_CLOSE, kernel)

    # --- Contour Detection ---

    # Find all external contours from the combined mask
    contours, _ = cv2.findContours(mask_combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("No plant detected.")
        return None

    # Assume the largest contour corresponds to the plant
    largest_contour = max(contours, key=cv2.contourArea)

    # Initialize variables for topmost and bottommost Y-coordinates
    top_y = np.inf
    bottom_y = 0

    # Loop through contour points to find topmost point (smallest Y)
    for point in largest_contour:
        x, y = point[0]
        if y < top_y:
            top_y = y

    # --- Bottom Detection Using Column Sum ---

    # To reduce false detections near soil, sum each row's pixels and scan from bottom up
    height, width = mask_combined.shape
    column_sums = np.sum(mask_combined, axis=1)

    for y in range(height - 1, 0, -1):
        if column_sums[y] > 1000:  # Threshold may be tuned per image size/content
            bottom_y = y
            break

    # Calculate plant height in pixels
    plant_height_px = bottom_y - top_y

    # Convert height to centimeters using reference object scaling
    pixels_per_cm = reference_pixel_height / reference_height_cm
    plant_height_cm = plant_height_px / pixels_per_cm

    # --- Visualization ---

    # Draw a vertical line and height label on the image for reference
    cv2.line(frame, (100, top_y), (100, bottom_y), (0, 255, 0), 2)
    cv2.putText(frame, f'Height: {plant_height_cm:.2f} cm', (120, top_y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    # Show intermediate and final outputs
    cv2.imshow("Original Frame with Height", frame)
    cv2.imshow("HSV Color Mask", mask_color)
    cv2.imshow("Thresholded Mask (Binary)", mask_thresh)
    cv2.imshow("Adaptive Threshold (Gray)", adaptive_thresh)
    cv2.imshow("Canny Edge Detection", edges)
    cv2.imshow("Combined Mask", mask_combined)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    print(f"Detected Plant Height: {plant_height_px} pixels ({plant_height_cm:.2f} cm)")
    return plant_height_px, plant_height_cm

# --- Example Usage ---

image_path = "plant_images/day9.1S.jpg"
reference_height_cm = 10
reference_pixel_height = 250

get_plant_height(image_path, reference_height_cm, reference_pixel_height)

Detected Plant Height: 168 pixels (6.72 cm)


(168, 6.72)

In [3]:
import cv2
import numpy as np

def count_leaves(image_path, min_contour_area=50):
    # Load image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (600, 600))  # Resize for consistency
    
    # Convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define green color range for plant
    lower_green = np.array([30, 40, 10])  
    upper_green = np.array([110, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Apply binary threshold to the mask (optional but can improve results)
    _, thresholded_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # Define a kernel for erosion and apply it
    kernel = np.ones((5, 5), np.uint8)
    eroded_mask = cv2.erode(thresholded_mask, kernel, iterations=3)

    # Apply morphological operations (closing and opening)
    eroded_mask = cv2.morphologyEx(eroded_mask, cv2.MORPH_CLOSE, kernel)
    eroded_mask = cv2.morphologyEx(eroded_mask, cv2.MORPH_OPEN, kernel)

    # Find contours
    contours, _ = cv2.findContours(eroded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours based on minimum area
    valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
    
    leaf_count = len(valid_contours)
    
    # Draw contours (bounding boxes around detected leaves)
    for cnt in valid_contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.putText(image, f'Leaves: {leaf_count}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Show images
    cv2.imshow("Leaf Count", image)
    cv2.imshow("Thresholded Mask", thresholded_mask)
    cv2.imshow("Eroded Mask", eroded_mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    print(f'Total Leaves Counted: {leaf_count}')
    return leaf_count

# Run the function with a minimum contour area
image_path = "plant_images/day9.1T.jpg"  # Provide actual image path
min_area = 50  # Minimum area for a valid contour
count_leaves(image_path, min_contour_area=min_area)

Total Leaves Counted: 4


4

In [8]:
import cv2
import numpy as np

def detect_health_issues(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not read the image.")
        return "Error"
    
    # Resize image for consistency
    image_resized = cv2.resize(image, (600, 400))
    
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image_resized, cv2.COLOR_BGR2HSV)
    
    # Define the range for healthy green color (leaves)
    lower_green = np.array([38, 45, 115])
    upper_green = np.array([90, 255, 255])
    
    # Mask for healthy leaves (green regions)
    green_mask = cv2.inRange(hsv, lower_green, upper_green)
    
    # Define the range for unhealthy yellow/brown color (indicating stress or disease)
    yellow_lower = np.array([20, 100, 100])
    yellow_upper = np.array([30, 255, 255])
    
    # Mask for unhealthy yellow regions
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)
    
    # Find contours in the green mask (healthy areas)
    green_contours, _ = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    healthy_area = sum(cv2.contourArea(cnt) for cnt in green_contours)
    
    # Find contours in the yellow mask (unhealthy areas)
    yellow_contours, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    unhealthy_area = sum(cv2.contourArea(cnt) for cnt in yellow_contours)
    
    # Calculate the total area of interest (total leaf area)
    total_area = healthy_area + unhealthy_area
    
    # Set thresholds for determining health based on the area of unhealthy regions
    health_status = "Healthy"
    if unhealthy_area > 0.1 * total_area:  # If more than 10% of the leaf area is unhealthy
        health_status = "Unhealthy"
    
    # Display results on the image
    cv2.putText(image_resized, f'Health: {health_status}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Show the images
    cv2.imshow("Original Image", image_resized)
    cv2.imshow("Green Mask (Healthy)", green_mask)
    cv2.imshow("Yellow Mask (Unhealthy)", yellow_mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return health_status

# Example Usage
health_status = detect_health_issues("plant_images/day9.1T.jpg")  # Replace with the actual path
print("Plant Health Status:", health_status)

Plant Health Status: Healthy


In [9]:
import cv2
import numpy as np
import os

def get_plant_height(image_path):
    """ Detect plant height in pixels from the given image with improved edge detection """
    frame = cv2.imread(image_path)

    if frame is None:
        print(f"Error loading image: {image_path}")
        return None

    # Resize to a consistent size (800x600)
    frame = cv2.resize(frame, (800, 600))

    # Convert to HSV
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define green color range
    lower_color = np.array([20, 40, 40])  
    upper_color = np.array([80, 255, 255])

    # Create color mask
    mask_color = cv2.inRange(img_hsv, lower_color, upper_color)

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)

    # Convert to grayscale for edge detection
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

    # Apply Canny Edge Detection
    edges = cv2.Canny(gray, 50, 150)

    # Combine Color Mask and Edge Mask using bitwise OR
    combined_mask = cv2.bitwise_or(mask_color, edges)

    # Morphological operations to remove noise
    kernel = np.ones((5, 5), np.uint8)
    processed_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    processed_mask = cv2.morphologyEx(processed_mask, cv2.MORPH_OPEN, kernel)

    # Find contours
    contours, _ = cv2.findContours(processed_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    min_contour_area = 50  # Minimum area to detect plant
    max_height = 0

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > min_contour_area:
            _, y, _, h = cv2.boundingRect(cnt)
            max_height = max(max_height, h)

    if max_height > 0:
        print(f"Detected Height in {image_path}: {max_height}px")
    else:
        print(f"No plant detected in {image_path}")
    
    # Show intermediate processing results
    cv2.imshow("Detected Edges", edges)
    cv2.imshow("Final Processed Mask", processed_mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return max_height if max_height > 0 else None


def calculate_growth_rate(image_day_old, image_day_new, days_between):
    """ Compute growth rate between two days """
    height_old = get_plant_height(image_day_old)
    height_new = get_plant_height(image_day_new)

    if height_old is None or height_new is None:
        print("Could not calculate growth rate due to missing height data.")
        return None

    growth_rate = (height_new - height_old) / days_between
    print(f"Growth Rate: {growth_rate:.2f} pixels/day")
    return growth_rate


# Example Usage:
image_day_1 = "plant_images/day1.1S.jpg"  # Image from earlier day
image_day_9 = "plant_images/day9.1S.jpg"  # Image from later day
days_between = 8  # Difference in days

calculate_growth_rate(image_day_1, image_day_9, days_between)

Detected Height in plant_images/day1.1S.jpg: 71px
Detected Height in plant_images/day9.1S.jpg: 169px
Growth Rate: 12.25 pixels/day


12.25